In [1]:
# Base User class
class User:
    def __init__(self, user_id, name):
        self.user_id = user_id
        self.name = name
    
    def get_details(self):
        return f"User ID: {self.user_id}, Name: {self.name}"

# Derived Customer class
# Customer Class

class Customer(User):
    def __init__(self, user_id, name, address):
        super().__init__(user_id, name)
        self.address = address
        self.cart = ShoppingCart()
        self.current_order = None  # Track the current order for payment

    def add_to_cart(self, product):
        if self.current_order and self.current_order.paid:
            print("Order has been paid for. No changes allowed.")
            return
        self.cart.add_product(product)
        print(f"{self.name} added {product.name} to the cart")

    def remove_from_cart(self, product):
        if self.current_order and self.current_order.paid:
            print("Order has been paid for. No changes allowed.")
            return
        self.cart.remove_product(product)
        print(f"{self.name} removed {product.name} from the cart")

    def place_order(self, store):
        if self.current_order and self.current_order.paid:
            print("Order has already been placed and paid for. No changes allowed.")
            return

        order = self.cart.checkout(store, self.address)
        if order:
            self.current_order = order
            print(f"Order placed by {self.name} for {order.get_order_details()}")

    def pay_for_order(self):
        # Check if the customer has an order to pay for
        if not self.current_order:
            print("No order to pay for.")
        # Check if the order is already paid
        elif self.current_order.paid:
            print("Order has already been paid.")
        # Proceed to make the payment
        else:
            # Display the total amount before payment
            print(f"Your total amount to pay is: ${self.current_order.total_cost:.2f}")
            confirm_payment = input("Do you want to proceed with the payment? (yes/no): ").lower()
            
            if confirm_payment == 'yes':
                # Simulate payment process
                self.current_order.pay()
            else:
                print("Payment canceled.")


# Derived Admin class
class Admin(User):
    def __init__(self, user_id, name):
        super().__init__(user_id, name)
        self.store_locked = False  # Track if store modifications are locked

    def add_product(self, product, store):
        if self.store_locked:
            print("Changes are locked after payment. No product can be added.")
            return
        store.add_product(product)
        print(f"{self.name} added {product.name} to the store")

    def remove_product(self, product, store):
        if self.store_locked:
            print("Changes are locked after payment. No product can be removed.")
            return
        store.remove_product(product)
        print(f"{self.name} removed {product.name} from the store")

# Product class
class Product:
    def __init__(self, product_id, name, price, stock):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.stock = stock

    def update_stock(self, quantity):
        self.stock += quantity

# Shopping Cart class
class ShoppingCart:
    def __init__(self):
        self.items = []

    def add_product(self, product):
        self.items.append(product)

    def remove_product(self, product):
        if product in self.items:
            self.items.remove(product)

    def checkout(self, store, address):
        if not self.items:
            print("The cart is empty. Add items to checkout.")
            return None
        
        total_cost = sum([item.price for item in self.items])
        order = Order(self.items, total_cost, address)
        store.process_order(order)
        self.items = []  # Empty the cart after checkout
        return order

# Order class
class Order:
    def __init__(self, items, total_cost, address):
        self.items = items
        self.total_cost = total_cost
        self.address = address
        self.paid = False  # New attribute to track payment status

    def get_order_details(self):
        item_names = [item.name for item in self.items]
        return f"Items: {', '.join(item_names)}, Total Cost: ${self.total_cost}"

    def pay(self):
        self.paid = True
        print(f"Order has been paid for. Total: ${self.total_cost}")

# Online Store class
class OnlineStore:
    def __init__(self):
        self.products = []
        self.orders = []

    def add_product(self, product):
        self.products.append(product)

    def remove_product(self, product):
        self.products = [p for p in self.products if p.product_id != product.product_id]

    def display_products(self):
        print("Available Products:")
        for product in self.products:
            print(f"ID: {product.product_id}, Name: {product.name}, Price: ${product.price}, Stock: {product.stock}")

    def process_order(self, order):
        # Deduct the product stock for each ordered item
        for product in order.items:
            product.update_stock(-1)
        self.orders.append(order)
        print("Order processed.")

    def lock_store(self):
        """Lock the store from changes after a payment has been made."""
        print("Checking if store should be locked due to payment...")
        for order in self.orders:
            if order.paid:
                print("Store is locked after payment. No more changes allowed.")
                return True
        return False

# Function to display a menu for user 
def display_menu():
    print("\nWelcome to the Online Store")             #admin can add products
    print("1. Admin: Add a Product")
    print("2. Admin: Remove a Product")                  # admin can remove products
    print("3. Customer: Add Product to Cart")
    print("4. Customer: Remove Product from Cart")
    print("5. Customer: Place Order")
    print("6. Customer: Pay for Order")
    print("7. Display Products")
    print("8. Exit")

# Function to add products via input product details
def create_product():
    product_id = int(input("Enter Product ID: "))               #admin enters product id 
    name = input("Enter Product Name: ")                         #product name
    price = float(input("Enter Product Price: "))
    stock = int(input("Enter Product Stock: "))
    return Product(product_id, name, price, stock)

# Demonstration of the Online Shopping System
store = OnlineStore()

# Create pre-added products
predefined_products = [
    Product(1, "Laptop", 1200, 10),
    Product(2, "Smartphone", 800, 15),
    Product(3, "Tablet", 300, 25),
    Product(4, "Headphones", 150, 40),
    Product(5, "Smartwatch", 250, 20)
]

# Add predefined products to the store
for product in predefined_products:
    store.add_product(product)

# Create users
admin = Admin(1, "Admin1")
customer = Customer(2, "Customer1", "123 Elm Street, Springfield")

# Main loop to handle user interactions
while True:
    display_menu()
    choice = int(input("Select an option: "))

    # Check if any order has been paid and lock store if necessary
    if store.lock_store():
        admin.store_locked = True

    if choice == 1:                 
        # Admin adds a product
        product = create_product()
        admin.add_product(product, store)
    
    elif choice == 2:
        # Admin removes a product
        store.display_products()
        product_id = int(input("Enter the Product ID to remove: "))
        product_to_remove = next((p for p in store.products if p.product_id == product_id), None)
        if product_to_remove:
            admin.remove_product(product_to_remove, store)
        else:
            print("Product not found.")
    
    elif choice == 3:
        # Customer adds a product to the cart
        store.display_products()
        product_id = int(input("Enter the Product ID to add to cart: "))
        product_to_add = next((p for p in store.products if p.product_id == product_id), None)
        if product_to_add:
            customer.add_to_cart(product_to_add)
        else:
            print("Product not found.")
    
    elif choice == 4:
        # Customer removes a product from the cart
        product_id = int(input("Enter the Product ID to remove from cart: "))
        product_to_remove = next((p for p in customer.cart.items if p.product_id == product_id), None)
        if product_to_remove:
            customer.remove_from_cart(product_to_remove)
        else:
            print("Product not found in cart.")
    
    elif choice == 5:
        # Customer places the order
        customer.place_order(store)

    elif choice == 6:
        # Customer pays for the order
        customer.pay_for_order()

    elif choice == 7:
        # Display products
        store.display_products()

    elif choice == 8:
        # Exit the program
        print("Exiting the store. Goodbye!")
        break

    else:
        print("Invalid option. Please try again.")



Welcome to the Online Store
1. Admin: Add a Product
2. Admin: Remove a Product
3. Customer: Add Product to Cart
4. Customer: Remove Product from Cart
5. Customer: Place Order
6. Customer: Pay for Order
7. Display Products
8. Exit


Select an option:  3


Checking if store should be locked due to payment...
Available Products:
ID: 1, Name: Laptop, Price: $1200, Stock: 10
ID: 2, Name: Smartphone, Price: $800, Stock: 15
ID: 3, Name: Tablet, Price: $300, Stock: 25
ID: 4, Name: Headphones, Price: $150, Stock: 40
ID: 5, Name: Smartwatch, Price: $250, Stock: 20


Enter the Product ID to add to cart:  1


Customer1 added Laptop to the cart

Welcome to the Online Store
1. Admin: Add a Product
2. Admin: Remove a Product
3. Customer: Add Product to Cart
4. Customer: Remove Product from Cart
5. Customer: Place Order
6. Customer: Pay for Order
7. Display Products
8. Exit


...

Checking if store should be locked due to payment...
Your total amount to pay is: $1200.00


Do you want to proceed with the payment? (yes/no):  yes


Order has been paid for. Total: $1200

Welcome to the Online Store
1. Admin: Add a Product
2. Admin: Remove a Product
3. Customer: Add Product to Cart
4. Customer: Remove Product from Cart
5. Customer: Place Order
6. Customer: Pay for Order
7. Display Products
8. Exit


Select an option:  8


Checking if store should be locked due to payment...
Store is locked after payment. No more changes allowed.
Exiting the store. Goodbye!


In [5]:
# Customer Class


